In [ ]:
# change to code to  save the storage space .Earlier it used to calculate frames for all the video and then after storing it .It used to give all the frame for each video to get features.npy for each video. frame for 400 video of 2.1GB tool 76GB of space .hence it will not be  feasible use for larger dataset of 1000 or 10k video

# hence instead of doing ela for all video and giving input to CNN we Instead of processing multiple ELA images, we now only process the 'error_level_plot.png' for each video folder.We extract features from this single image and save them as 'features.npy'.


In [1]:
import os
import cv2
import numpy as np
from PIL import Image, ImageChops, ImageEnhance
import matplotlib.pyplot as plt

def resize_frame(frame, scale_factor=0.5):
    height, width = frame.shape[:2]
    new_size = (int(width * scale_factor), int(height * scale_factor))
    resized_frame = cv2.resize(frame, new_size, interpolation=cv2.INTER_LANCZOS4)
    return resized_frame

def apply_ela(image_path, scale_factor=0.5):
    original = Image.open(image_path)
    original = original.convert("RGB")

    resized = original.resize((int(original.width * scale_factor), int(original.height * scale_factor)), Image.Resampling.LANCZOS)

    original_np = np.array(original)
    resized_np = np.array(resized)

    error_level = ImageChops.difference(Image.fromarray(original_np), Image.fromarray(resized_np))

    error_level_np = np.array(error_level)

    mean_error = np.mean(error_level_np)

    return mean_error

def process_videos(video_folder, output_folder):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    video_files = [os.path.join(video_folder, f) for f in os.listdir(video_folder) if f.lower().endswith(('.mp4', '.avi', '.mov', '.mkv'))]

    for video_file in video_files:
        video_name = os.path.splitext(os.path.basename(video_file))[0]
        video_output_folder = os.path.join(output_folder, video_name)
        if not os.path.exists(video_output_folder):
            os.makedirs(video_output_folder)

        cap = cv2.VideoCapture(video_file)

        if not cap.isOpened():
            print(f"Error: Could not open video file {video_file}")
            continue

        frame_number = 0
        mean_errors = []

        while True:
            ret, frame = cap.read()
            if not ret:
                break

            resized_frame = resize_frame(frame)

            frame_filename = os.path.join(video_output_folder, f"frame_{frame_number}.png")
            cv2.imwrite(frame_filename, resized_frame)

            mean_error = apply_ela(frame_filename)
            mean_errors.append(mean_error)

            os.remove(frame_filename)  # Delete the frame image after processing

            frame_number += 1

        cap.release()
        print(f"Processing complete for {video_file}")

        plt.figure(figsize=(10, 5))
        plt.plot(mean_errors, label=f'Error Level - {video_name}')
        plt.xlabel('Frame Number')
        plt.ylabel('Mean Error Level')
        plt.title(f'Mean Error Level Across Frames for {video_name}')
        plt.legend()
        plt.grid(True)
        plt.savefig(os.path.join(video_output_folder, 'error_level_plot.png'))
        plt.close()

        # Save mean_errors as a numpy array
        np.save(os.path.join(video_output_folder, 'mean_errors.npy'), np.array(mean_errors))

# Example usage
video_folder = "train_sample_videos"
output_folder = "output_frames1"

process_videos(video_folder, output_folder)

Processing complete for train_sample_videos/cdaxixbosp.mp4
Processing complete for train_sample_videos/btiysiskpf.mp4
Processing complete for train_sample_videos/clihsshdkq.mp4
Processing complete for train_sample_videos/alvgwypubw.mp4
Processing complete for train_sample_videos/eqvuznuwsa.mp4
Processing complete for train_sample_videos/eudeqjhdfd.mp4
Processing complete for train_sample_videos/eeyhxisdfh.mp4
Processing complete for train_sample_videos/cizlkenljw.mp4
Processing complete for train_sample_videos/bndybcqhfr.mp4
Processing complete for train_sample_videos/cuzrgrbvil.mp4
Processing complete for train_sample_videos/atyntldecu.mp4
Processing complete for train_sample_videos/bggsurpgpr.mp4
Processing complete for train_sample_videos/eckvhdusax.mp4
Processing complete for train_sample_videos/dvakowbgbt.mp4
Processing complete for train_sample_videos/dqqtjcryjv.mp4
Processing complete for train_sample_videos/djvutyvaio.mp4
Processing complete for train_sample_videos/dzwkmcwkwl.m

KeyboardInterrupt: 

In [7]:
# modified above code to skip the file which has already been processed

import os
import cv2
import numpy as np
from PIL import Image, ImageChops, ImageEnhance
import matplotlib.pyplot as plt

def resize_frame(frame, scale_factor=0.5):
    height, width = frame.shape[:2]
    new_size = (int(width * scale_factor), int(height * scale_factor))
    resized_frame = cv2.resize(frame, new_size, interpolation=cv2.INTER_LANCZOS4)
    return resized_frame

def apply_ela(image_path, scale_factor=0.5):
    original = Image.open(image_path)
    original = original.convert("RGB")#L FOR TRANSLATING COLOR IMAGE TO GREYSCALE

    resized = original.resize((int(original.width * scale_factor), int(original.height * scale_factor)), Image.Resampling.LANCZOS)

    original_np = np.array(original)
    resized_np = np.array(resized)

    error_level = ImageChops.difference(Image.fromarray(original_np), Image.fromarray(resized_np))

    error_level_np = np.array(error_level)

    mean_error = np.mean(error_level_np)

    return mean_error

def process_videos(video_folder, output_folder):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    video_files = [os.path.join(video_folder, f) for f in os.listdir(video_folder) if f.lower().endswith(('.mp4', '.avi', '.mov', '.mkv'))]

    for video_file in video_files:
        video_name = os.path.splitext(os.path.basename(video_file))[0]
        video_output_folder = os.path.join(output_folder, video_name)

        # Skip processing if mean_errors.npy already exists
        if os.path.exists(os.path.join(video_output_folder, 'mean_errors.npy')):
            print(f"Skipping {video_file} as it has already been processed.")
            continue

        if not os.path.exists(video_output_folder):
            os.makedirs(video_output_folder)

        cap = cv2.VideoCapture(video_file)

        if not cap.isOpened():
            print(f"Error: Could not open video file {video_file}")
            continue

        frame_number = 0
        mean_errors = []

        while True:
            ret, frame = cap.read()
            if not ret:
                break

            resized_frame = resize_frame(frame)

            frame_filename = os.path.join(video_output_folder, f"frame_{frame_number}.png")
            cv2.imwrite(frame_filename, resized_frame)

            mean_error = apply_ela(frame_filename)
            mean_errors.append(mean_error)

            os.remove(frame_filename)  # Delete the frame image after processing

            frame_number += 1

        cap.release()
        print(f"Processing complete for {video_file}")

        plt.figure(figsize=(10, 5))
        plt.plot(mean_errors, label=f'Error Level - {video_name}')
        plt.xlabel('Frame Number')
        plt.ylabel('Mean Error Level')
        plt.title(f'Mean Error Level Across Frames for {video_name}')
        plt.legend()
        plt.grid(True)
        plt.savefig(os.path.join(video_output_folder, 'error_level_plot.png'))
        plt.close()

        # Save mean_errors as a numpy array
        np.save(os.path.join(video_output_folder, 'mean_errors.npy'), np.array(mean_errors))

# Example usage
video_folder = "train_sample_videos"
output_folder = "output_frames1"

process_videos(video_folder, output_folder)


Skipping train_sample_videos/cdaxixbosp.mp4 as it has already been processed.
Skipping train_sample_videos/btiysiskpf.mp4 as it has already been processed.
Skipping train_sample_videos/clihsshdkq.mp4 as it has already been processed.
Skipping train_sample_videos/alvgwypubw.mp4 as it has already been processed.
Skipping train_sample_videos/eqvuznuwsa.mp4 as it has already been processed.
Skipping train_sample_videos/eudeqjhdfd.mp4 as it has already been processed.
Skipping train_sample_videos/eeyhxisdfh.mp4 as it has already been processed.
Skipping train_sample_videos/cizlkenljw.mp4 as it has already been processed.
Skipping train_sample_videos/bndybcqhfr.mp4 as it has already been processed.
Skipping train_sample_videos/cuzrgrbvil.mp4 as it has already been processed.
Skipping train_sample_videos/atyntldecu.mp4 as it has already been processed.
Skipping train_sample_videos/bggsurpgpr.mp4 as it has already been processed.
Skipping train_sample_videos/eckvhdusax.mp4 as it has already be

In [8]:
# USE PRETRAINED MODEL RESNET18 TO EXTRACT FEATURE
import os
import torch
import torchvision.models as models
import torchvision.transforms as transforms
from PIL import Image
import numpy as np

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

model = models.resnet18(pretrained=True)
model = torch.nn.Sequential(*list(model.children())[:-1])
model.eval()

def extract_features(image_path):
    try:
        image = Image.open(image_path).convert("RGB")
        image_tensor = transform(image).unsqueeze(0)

        print(f"Processing image: {image_path}")
        print(f"Image tensor shape: {image_tensor.shape}")

        with torch.no_grad():
            features = model(image_tensor)
            print(f"Extracted features shape: {features.shape}")

        features = features.squeeze().numpy()
        print(f"Squeezed features shape: {features.shape}")
        return features
    except Exception as e:
        print(f"Error processing {image_path}: {e}")
        return None

def process_video_folders(base_folder):
    for video_folder in os.listdir(base_folder):
        video_folder_path = os.path.join(base_folder, video_folder)
        if not os.path.isdir(video_folder_path):
            continue

        print(f"Processing video folder: {video_folder}")

        error_level_plot = os.path.join(video_folder_path, 'error_level_plot.png')

        if os.path.exists(error_level_plot):
            features = extract_features(error_level_plot)
            if features is not None and features.size > 0:
                features_output_path = os.path.join(video_folder_path, 'features.npy')
                np.save(features_output_path, features)
                print(f"Features saved for {video_folder}")
            else:
                print(f"No features were extracted for {video_folder}")
        else:
            print(f"Error level plot not found for {video_folder}")

# Example usage
base_folder = "output_frames1"
process_video_folders(base_folder)

/opt/anaconda3/envs/Implementation/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/anaconda3/envs/Implementation/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Processing video folder: ahqqqilsxt
Processing image: output_frames1/ahqqqilsxt/error_level_plot.png
Image tensor shape: torch.Size([1, 3, 224, 224])
Extracted features shape: torch.Size([1, 512, 1, 1])
Squeezed features shape: (512,)
Features saved for ahqqqilsxt
Processing video folder: djvtbgwdcc
Processing image: output_frames1/djvtbgwdcc/error_level_plot.png
Image tensor shape: torch.Size([1, 3, 224, 224])
Extracted features shape: torch.Size([1, 512, 1, 1])
Squeezed features shape: (512,)
Features saved for djvtbgwdcc
Processing video folder: atzdznmder
Processing image: output_frames1/atzdznmder/error_level_plot.png
Image tensor shape: torch.Size([1, 3, 224, 224])
Extracted features shape: torch.Size([1, 512, 1, 1])
Squeezed features shape: (512,)
Features saved for atzdznmder
Processing video folder: esyrimvzsa
Processing image: output_frames1/esyrimvzsa/error_level_plot.png
Image tensor shape: torch.Size([1, 3, 224, 224])
Extracted features shape: torch.Size([1, 512, 1, 1])
Sq

In [11]:
# ORAGANIZED THE FEATURES.NPY OF EACH VIDEO ACCORDING TO THERE LABEL FROM METADATA.JSON FILE 
import json
import os
import shutil

# Define paths
input_frames_dir = "output_frames1"
output_dir = "organized_frames1"
metadata_file = "metadata.json"

# Create output directories
os.makedirs(os.path.join(output_dir, "REAL"), exist_ok=True)
os.makedirs(os.path.join(output_dir, "FAKE"), exist_ok=True)

# Read metadata
with open(metadata_file, "r") as f:
    metadata = json.load(f)

# Process each video
for video, info in metadata.items():
    label = info["label"]
    video_name = os.path.splitext(video)[0]  # Remove the .mp4 extension

    # Source and destination paths
    src_path = os.path.join(input_frames_dir, video_name)
    dst_path = os.path.join(output_dir, label, video_name)

    # Check if the source directory exists
    if os.path.exists(src_path):
        # Copy the frame folder to the appropriate category
        shutil.copytree(src_path, dst_path)
        print(f"Copied {video_name} to {label} category")
    else:
        print(f"Warning: Frame folder for {video_name} not found")

print("Organization complete!")

Copied aagfhgtpmv to FAKE category
Copied aapnvogymq to FAKE category
Copied abarnvbtwb to REAL category
Copied abofeumbvv to FAKE category
Copied abqwwspghj to FAKE category
Copied acifjvzvpm to FAKE category
Copied acqfdwsrhi to FAKE category
Copied acxnxvbsxk to FAKE category
Copied acxwigylke to FAKE category
Copied aczrgyricp to FAKE category
Copied adhsbajydo to FAKE category
Copied adohikbdaz to FAKE category
Copied adylbeequz to FAKE category
Copied aelfnikyqj to REAL category
Copied aelzhcnwgf to FAKE category
Copied aettqgevhz to FAKE category
Copied aevrfsexku to FAKE category
Copied afoovlsmtx to REAL category
Copied agdkmztvby to FAKE category
Copied agqphdxmwt to FAKE category
Copied agrmhtjdlk to REAL category
Copied ahbweevwpv to FAKE category
Copied ahdbuwqxit to FAKE category
Copied ahfazfbntc to FAKE category
Copied ahqqqilsxt to REAL category
Copied aipfdnwpoo to FAKE category
Copied ajqslcypsw to REAL category
Copied ajwpjhrbcv to FAKE category
Copied aklqzsddfl to

In [12]:
# DID CLASSIFICATION USING SVM AND KNN
import os
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report
from tqdm import tqdm

# Define paths
data_dir = "organized_frames1"
categories = ["FAKE", "REAL"]

# Initialize data lists
features = []
labels = []

# Load features and labels
for category in categories:
    category_path = os.path.join(data_dir, category, "")
    label = categories.index(category)
    print(f"Processing {category} videos...")

    for video_folder in tqdm(os.listdir(category_path)):
        video_path = os.path.join(category_path, video_folder)
        features_path = os.path.join(video_path, "features.npy")

        if os.path.exists(features_path):
            video_features = np.load(features_path)
            features.append(video_features)
            labels.append(label)
        else:
            print(f"Warning: features.npy not found in {video_path}")

# Convert lists to numpy arrays
features = np.array(features)
labels = np.array(labels)

# Ensure features and labels are not empty
if len(features) == 0:
    raise ValueError("Features array is empty. Please check the data loading process.")

# Flatten the features if necessary
if len(features.shape) > 2:
    features = features.reshape(features.shape[0], -1)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)

# SVM Model
print("Training SVM model...")
svm_model = SVC(kernel='sigmoid')  # You can experiment with different kernels like 'rbf'
svm_model.fit(X_train, y_train)
y_pred_svm = svm_model.predict(X_test)

# Evaluate SVM
print("SVM Classification Report:")
print(classification_report(y_test, y_pred_svm))
print(f"SVM Accuracy: {accuracy_score(y_test, y_pred_svm):.2f}")

# KNN Model
print("Training KNN model...")
knn_model = KNeighborsClassifier(n_neighbors=4)  # You can adjust the number of neighbors
knn_model.fit(X_train, y_train)
y_pred_knn = knn_model.predict(X_test)

# Evaluate KNN
print("KNN Classification Report:")
print(classification_report(y_test, y_pred_knn))
print(f"KNN Accuracy: {accuracy_score(y_test, y_pred_knn):.2f}")

Processing FAKE videos...


100%|██████████| 323/323 [00:00<00:00, 10324.11it/s]


Processing REAL videos...


100%|██████████| 77/77 [00:00<00:00, 10847.82it/s]

Training SVM model...
SVM Classification Report:
              precision    recall  f1-score   support

           0       0.80      1.00      0.89        64
           1       0.00      0.00      0.00        16

    accuracy                           0.80        80
   macro avg       0.40      0.50      0.44        80
weighted avg       0.64      0.80      0.71        80

SVM Accuracy: 0.80
Training KNN model...
KNN Classification Report:
              precision    recall  f1-score   support

           0       0.82      0.98      0.89        64
           1       0.67      0.12      0.21        16

    accuracy                           0.81        80
   macro avg       0.74      0.55      0.55        80
weighted avg       0.79      0.81      0.76        80

KNN Accuracy: 0.81



/opt/anaconda3/envs/Implementation/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/envs/Implementation/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/envs/Implementation/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(res